In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
!pip install wget
import wget
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data=pd.read_csv('Tweets.csv')
data

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [ ]:
data.columns

Index(['textID', 'text', 'selected_text', 'sentiment'], dtype='object')

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

# Define the pre-processing function
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', str(text))
     # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if not word in stop_words]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    #stemming the tokens
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    return text


# Apply the pre-processing function to the 'text' column of the dataset
data['text'] = data['text'].apply(preprocess_text)

# Display the pre-processed data
print(data.head())

       textID                                    text  \
0  cb774db0d1                      id responded going   
1  549e992a42                 sooo sad miss san diego   
2  088c60f138                            bos bullying   
3  9642c003ef                   interview leave alone   
4  358bd9e861  son couldnt put release already bought   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  


In [ ]:
# Import the necessary libraries
!pip install --upgrade gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import nltk
import numpy as np

# Split the data into training and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['sentiment'], test_size=0.2, random_state=42)

# Bag-of-Words feature extraction
vectorizer_bow = CountVectorizer()
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)

# TF-IDF feature extraction
vectorizer_tfidf = TfidfVectorizer()
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

# Word Embeddings feature extraction
corpus = [nltk.word_tokenize(tweet) for tweet in X_train.tolist()]
model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)
X_train_w2v = np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(100)], axis=0) for words in corpus])

corpus = [nltk.word_tokenize(tweet) for tweet in X_test.tolist()]
X_test_w2v = np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(100)], axis=0) for words in corpus])



# Display the shape of the feature matrices
print('Bag-of-Words shape:', X_train_bow.shape)
print('TF-IDF shape:', X_train_tfidf.shape)
print('Word Embeddings shape:', X_train_w2v.shape)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Bag-of-Words shape: (21984, 22308)
TF-IDF shape: (21984, 22308)
Word Embeddings shape: (21984, 100)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [ ]:
# Apply Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_vect, y_train)
nb_pred = nb.predict(X_test_vect)
nb_acc = accuracy_score(y_test, nb_pred)
nb_prec=precision_score(y_test,nb_pred,average='weighted')
nb_rec=recall_score(y_test,nb_pred,average='weighted')
nb_F1=f1_score(y_test,nb_pred,average='weighted')

In [ ]:
# Apply SVM
svm = LinearSVC()
svm.fit(X_train_vect, y_train)
svm_pred = svm.predict(X_test_vect)
svm_acc = accuracy_score(y_test, svm_pred)
svm_prec=precision_score(y_test,svm_pred,average='weighted')
svm_rec=recall_score(y_test,svm_pred,average='weighted')
svm_F1=f1_score(y_test,svm_pred,average='weighted')

In [ ]:
# Apply Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_vect, y_train)
lr_pred = lr.predict(X_test_vect)
lr_acc = accuracy_score(y_test, lr_pred)
lr_prec=precision_score(y_test,lr_pred,average='weighted')
lr_rec=recall_score(y_test,lr_pred,average='weighted')
lr_F1=f1_score(y_test,lr_pred,average='weighted')

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Print the accuracies
print("Naive Bayes Accuracy:", nb_acc)
print("SVM Accuracy:", svm_acc)
print("Logistic Regression Accuracy:", lr_acc)
#print("Random Forest Accuracy:", rf_acc)

Naive Bayes Accuracy: 0.6214298708386392
SVM Accuracy: 0.6707294888120793
Logistic Regression Accuracy: 0.6891031471711843


In [ ]:
#print recall
print("Naive BAyes recall:",nb_rec)
print("SVM recall:",svm_rec)
print("logistic regression:",lr_rec)

Naive BAyes recall: 0.6214298708386392
SVM recall: 0.6707294888120793
logistic regression: 0.6891031471711843


In [ ]:
#print precision
print("NAive bayes precision:",nb_prec)
print("SVM  precision:",svm_prec)
print("logistic regression precision:",lr_prec)

NAive bayes precision: 0.6715879507813327
SVM  precision: 0.6721127021211178
logistic regression precision: 0.6979059547828015


In [ ]:
#print f1 score
print("Naive bayes f1 score:",nb_F1)
print("SVM f1 score:",svm_F1)
print("Logistic regression f1 score:",lr_F1)

Naive bayes f1 score: 0.6133725679629717
SVM f1 score: 0.670831392391178
Logistic regression f1 score: 0.6890539517306807


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd



# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['sentiment'], test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Define the parameter grid
param_grid = {
    'vect__max_features': [1000, 5000, 10000],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# Print the accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Best Hyperparameters: {'clf__C': 1, 'vect__max_features': 5000, 'vect__ngram_range': (1, 2)}
Accuracy: 0.6830998726578134


In [ ]:
import torch
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def predict(text, include_neutral=True):
    # Tokenize text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    # Predict
    logits = model(input_ids, attention_mask).logits
    probs = torch.softmax(logits, dim=-1)
    score = probs.detach().cpu().numpy()[0]
    label = None  # Set a default value for label
    if include_neutral:
        if (score >= 0.4).all() and (score <= 0.6).all():
            label = "Neutral"
        elif (score < 0.4).all():
            label = "Negative"
        elif (score > 0.6).all():
            label = "Positive"
    else:
        if score < 0.5:
            label = "Negative"
        else:
            label = "Positive"
    return {"label": label, "score": score.tolist()}




In [ ]:
predict("you are beautiful")

TypeError: ignored

In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score
import pandas as pd


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['sentiment'], test_size=0.2, random_state=42)

# Load the pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenize the input text
X_train_tokens = tokenizer.batch_encode_plus(X_train.tolist(), max_length=128, padding=True, truncation=True, return_tensors='pt')
X_test_tokens = tokenizer.batch_encode_plus(X_test.tolist(), max_length=128, padding=True, truncation=True, return_tensors='pt')

# Convert the labels to tensors
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(3):
    model.train()
    optimizer.zero_grad()
    outputs = model(input_ids=X_train_tokens['input_ids'], attention_mask=X_train_tokens['attention_mask'], labels=y_train_tensor)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(input_ids=X_test_tokens['input_ids'], attention_mask=X_test_tokens['attention_mask'])
    logits = outputs.logits
    y_pred = torch.argmax(logits, dim=1)

# Print the accuracy
acc = accuracy_score(y_test_tensor, y_pred)
print("Accuracy:", acc)

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPooling1D, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
data = pd.read_csv("Tweets.csv", encoding='latin-1')

# Data cleaning and preprocessing
def clean_text(text):
    # Remove mentions and URLs
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered = [word for word in tokens if word not in stop_words]
    text = ' '.join(filtered)
    return text

data['text'] = data['text'].apply(clean_text)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

# Random Forest
# Vectorize the text data
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_data['text'])
test_vectors = vectorizer.transform(test_data['text'])
# Train the model
rfc = RandomForestClassifier()
rfc.fit(train_vectors, train_data['label'])
# Predict on the test data
rfc_preds = rfc.predict(test_vectors)
# Evaluate the model
rfc_acc = accuracy_score(test_data['label'], rfc_preds)
rfc_cm = confusion_matrix(test_data['label'], rfc_preds)

print("Random Forest Accuracy: {:.2f}%".format(rfc_acc*100))
print("Random Forest Confusion Matrix:\n", rfc_cm)